In [3]:
pip install opencv-python scikit-learn tensorflow

  Using cached tensorflow-2.11.0-cp39-cp39-win_amd64.whl (1.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import random
import pickle
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

In [2]:
df = pd.read_csv("/breeds_classification/dog-breed-identification/labels.csv")

In [3]:
train_file = '/breeds_classification/dog-breed-identification/train/'
test_file = '/breeds_classification/dog-breed-identification/test/'

In [4]:
df

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
...,...,...
10217,ffd25009d635cfd16e793503ac5edef0,borzoi
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher


In [6]:
num_breeds = 90
im_size = 224
batch_size = 128
encoder = LabelEncoder()

In [7]:
breeds_list = list(df['breed'].value_counts().keys()) 

In [9]:
breeds = []

while(len(breeds) < 90):
    breed = random.choice(breeds_list)
    if breed in breeds:
        pass
    else:
        breeds.append(breed)
        
df = df.query('breed in @breeds')

In [12]:
breeds

['papillon',
 'chihuahua',
 'flat-coated_retriever',
 'bluetick',
 'scottish_deerhound',
 'clumber',
 'english_setter',
 'old_english_sheepdog',
 'affenpinscher',
 'norfolk_terrier',
 'dingo',
 'malinois',
 'bernese_mountain_dog',
 'basenji',
 'sussex_spaniel',
 'briard',
 'siberian_husky',
 'leonberg',
 'chow',
 'mexican_hairless',
 'malamute',
 'brabancon_griffon',
 'pekinese',
 'saint_bernard',
 'giant_schnauzer',
 'greater_swiss_mountain_dog',
 'boston_bull',
 'schipperke',
 'soft-coated_wheaten_terrier',
 'appenzeller',
 'weimaraner',
 'scotch_terrier',
 'chesapeake_bay_retriever',
 'miniature_poodle',
 'ibizan_hound',
 'border_collie',
 'otterhound',
 'irish_water_spaniel',
 'australian_terrier',
 'vizsla',
 'toy_terrier',
 'pembroke',
 'irish_setter',
 'collie',
 'irish_terrier',
 'cairn',
 'beagle',
 'norwich_terrier',
 'kuvasz',
 'yorkshire_terrier',
 'norwegian_elkhound',
 'bull_mastiff',
 'toy_poodle',
 'saluki',
 'kerry_blue_terrier',
 'bloodhound',
 'bedlington_terrier',
 

In [13]:
df

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
...,...,...
10214,ffcb610e811817766085054616551f9c,briard
10216,ffcffab7e4beef9a9b8076ef2ca51909,samoyed
10217,ffd25009d635cfd16e793503ac5edef0,borzoi
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont


In [14]:
df['image'] = df['id'].apply(lambda x:x + ".jpg")

C:\Users\deepa\AppData\Local\Temp\ipykernel_1360\3603616619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image'] = df['id'].apply(lambda x:x + ".jpg")


In [15]:
df

,id,breed,image
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,000bec180eb18c7604dcecc8fe0dba07.jpg
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,001513dfcb2ffafc82cccf4d8bbaba97.jpg
2,001cdf01b096e06d78e9e5112d419397,pekinese,001cdf01b096e06d78e9e5112d419397.jpg
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,00214f311d5d2247d5dfe4fe24b2303d.jpg
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier,002211c81b498ef88e1b40b9abf84e1d.jpg
...,...,...,...
10214,ffcb610e811817766085054616551f9c,briard,ffcb610e811817766085054616551f9c.jpg
10216,ffcffab7e4beef9a9b8076ef2ca51909,samoyed,ffcffab7e4beef9a9b8076ef2ca51909.jpg
10217,ffd25009d635cfd16e793503ac5edef0,borzoi,ffd25009d635cfd16e793503ac5edef0.jpg
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont,ffd3f636f7f379c51ba3648a9ff8254f.jpg


In [16]:
train_x = np.zeros((len(df), im_size, im_size, 3), dtype = float)

In [17]:
train_x.shape

(7723, 224, 224, 3)

In [19]:
for i, img_id in enumerate(df['image']):
    readed_img = cv2.imread(train_file+img_id,cv2.IMREAD_COLOR)
    img = cv2.resize(readed_img,(224, 224)) 
    img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
    train_x[i] = img_array

In [20]:
train_y = encoder.fit_transform(df["breed"].values)

In [21]:
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=42)

In [22]:
train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow(x_train, 
                                     y_train, 
                                     batch_size=batch_size)
 
test_datagen = ImageDataGenerator()
 
test_generator = test_datagen.flow(x_test, 
                                     y_test, 
                                     batch_size=batch_size)

In [23]:
resnet = ResNet50V2(input_shape = [im_size,im_size,3], weights='imagenet', include_top=False)
for layer in resnet.layers:
    layer.trainable = False
x = resnet.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

94668760/94668760 [==============================] - 13s 0us/step


In [24]:
predictions = Dense(120, activation='softmax')(x)
model = Model(inputs=resnet.input, outputs=predictions)

In [25]:
epochs = 25
learning_rate = 1e-3
optimizer = RMSprop(learning_rate=learning_rate,rho=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])
model.fit(train_generator,steps_per_epoch= x_train.shape[0] // batch_size,epochs= epochs,validation_data= test_generator,
                 validation_steps= x_test.shape[0]// batch_size)


Epoch 1/25
48/48 [==============================] - 640s 13s/step - loss: 2.7117 - accuracy: 0.3805 - val_loss: 0.9702 - val_accuracy: 0.7311
Epoch 2/25
48/48 [==============================] - 698s 15s/step - loss: 1.5177 - accuracy: 0.5818 - val_loss: 0.7931 - val_accuracy: 0.7643
Epoch 3/25
48/48 [==============================] - 699s 15s/step - loss: 1.2977 - accuracy: 0.6314 - val_loss: 0.7547 - val_accuracy: 0.7734
Epoch 4/25
48/48 [==============================] - 623s 13s/step - loss: 1.1723 - accuracy: 0.6579 - val_loss: 0.7383 - val_accuracy: 0.7806
Epoch 5/25
48/48 [==============================] - 614s 13s/step - loss: 1.1147 - accuracy: 0.6707 - val_loss: 0.7420 - val_accuracy: 0.7669
Epoch 6/25
48/48 [==============================] - 617s 13s/step - loss: 1.0580 - accuracy: 0.6851 - val_loss: 0.7404 - val_accuracy: 0.7708
Epoch 7/25
48/48 [==============================] - 611s 13s/step - loss: 1.0366 - accuracy: 0.6911 - val_loss: 0.7283 - val_accuracy: 0.7669
Epoch 

In [26]:
model.save("model")

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [24]:

model = load_model("model")
 
    
pred_img_path = '/jupyter-notebooks/download.jpg'
pred_img_array = cv2.resize(cv2.imread(pred_img_path,cv2.IMREAD_COLOR),((im_size,im_size)))
pred_img_array = preprocess_input(np.expand_dims(np.array(pred_img_array[...,::-1].astype(np.float32)).copy(), axis=0))
 
pred_val = model.predict(np.array(pred_img_array,dtype="float32"))
 
pred_breed = sorted(breed_dict)[np.argmax(pred_val)]
print("Predicted Breed for this Dog is :",pred_breed)

1/1 [==============================] - 2s 2s/step
Predicted Breed for this Dog is : rottweiler


In [2]:
import os
UPLOAD_FOLDER = os.path.join('static', 'uploads')

In [3]:
UPLOAD_FOLDER

'static\\uploads'